### Queries na base de dados Mongo


In [2]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo
import yaml

In [8]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [6]:
user_posgres = cfg["postgres"]["user"]
passwd_posgres = cfg["postgres"]["passwd"]
host_posgres = cfg["postgres"]["host"]
dbmongo_posgres = cfg["postgres"]["db"]

In [9]:
host_posgres

'34.134.72.9'

In [3]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [4]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [6]:
col_processos.count_documents({})

48204

In [7]:
#df = pd.DataFrame.from_records(processos,index=['nro_processo','link_processo','nome_parte'])
dados = pd.DataFrame(list(col_processos.find({})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

(48204, 8)

In [8]:
# Processos por empresa
df_processos.groupby('cod_empresa')['nro_processo'].count()

cod_empresa
ALLIANZ         4315
AZUL            5682
EMS               91
GOL             4195
MAPFRE          5281
MLUIZA          1775
N/I               38
PORTOSEGURO    14436
SULAMERICA      3317
TAM             6239
TOKIOMARINE     2832
Name: nro_processo, dtype: int64

In [10]:
df_processos.columns

Index(['nro_processo', 'link_processo', 'nome_parte', 'classe_processo',
       'assunto_processo', 'data_local_distribuicao', 'cod_empresa',
       'tipo_participacao'],
      dtype='object')

In [11]:
from sqlalchemy import create_engine
import psycopg2

In [13]:
conn_string_postgres = ''
  
db = create_engine(conn_string_postgres)
conn = db.connect()

In [16]:
df_processos.to_sql('dw_processos', con=conn, if_exists='replace',index=False)

204

# Decisões

In [17]:
# Carrega base de decisões total
dados_decisoes = pd.DataFrame(list(col_decisoes.find({})))
df_decisoes = pd.json_normalize(dados_decisoes['decisao'])

In [18]:
df_decisoes.shape

(13850, 11)

In [19]:
df_decisoes.to_sql('dw_decisoes', con=conn, if_exists='replace',index=False)

850

In [13]:
df_final = pd.merge(df_processos,df_decisoes,on="nro_processo")

In [15]:
df_final.columns

Index(['nro_processo', 'link_processo', 'nome_parte', 'classe_processo',
       'assunto_processo', 'data_local_distribuicao', 'cod_empresa',
       'tipo_participacao', 'classe', 'assunto', 'magistrado', 'comarca',
       'foro', 'vara', 'data_disp', 'texto_decisao', 'instancia', 'resultado'],
      dtype='object')

In [17]:
df_final = df_final[['nro_processo', 'link_processo', 'nome_parte', 'classe_processo','assunto_processo', 
'data_local_distribuicao', 'cod_empresa','tipo_participacao', 'classe', 'assunto', 'magistrado', 'comarca',
'foro', 'vara', 'data_disp', 'instancia', 'resultado']]

In [20]:
df_final['tribunal'] = 'TJSP'
df_final['justica'] = 'Justiça Estadual'

In [21]:
df_final.to_excel('../../dados/analiticos/base_processos.xlsx',sheet_name="DADOS",index=False)

In [ ]:
col_decisoes.count_documents({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}})

In [ ]:
col_decisoes.update_many({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}},{'$set':{'resultado':'Parcialmente Procedente'}})

In [ ]:
decisao = col_decisoes.find_one({'decisao.texto_decisao':{'$regex':"( JULGO PARCIALMENTE PROCEDENTE)"}})
decisao['decisao']

In [ ]:
# Decisões por Magistrado
df_qtdedecisoes = pd.DataFrame({"decisoes" : df_decisoes.groupby('magistrado').size()}).reset_index()
df_qtdedecisoes.sort_values(by="decisoes",ascending=False)